In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import geopandas as gpd
import pandas as pd
from siuba import *
import numpy as np

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2
import folium
import itertools

In [49]:
from update_vars import (analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, AM_PEAK, PM_PEAK, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD, SHARED_STOP_THRESHOLD)

In [31]:
import create_aggregate_stop_frequencies as casf

In [41]:
import importlib
importlib.reload(create_aggregate_stop_frequencies)

<module 'create_aggregate_stop_frequencies' from '/home/jovyan/data-analyses/high_quality_transit_areas/create_aggregate_stop_frequencies.py'>

In [6]:
am_peak_hrs = list(range(AM_PEAK[0].hour, AM_PEAK[1].hour))
pm_peak_hrs = list(range(PM_PEAK[0].hour, PM_PEAK[1].hour))
both_peaks_hrs = am_peak_hrs + pm_peak_hrs

In [7]:
analysis_date

'2025-05-14'

In [8]:
stop_times = helpers.import_scheduled_stop_times(
    analysis_date,
    get_pandas = True,
)

In [9]:
stop_times = casf.add_route_dir(stop_times, analysis_date)

In [10]:
st_prepped = stop_times.pipe(casf.prep_stop_times)

/opt/conda/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


## HCD Y-Branching

* combine single route frequencies with collinearity screen...
* actually, these routes need to pass _both_ a collinearity screen and a non-collinearity screen: 8+ stops shared and also 8+ stops not shared (meaningful service on all 3 legs of the Y)

In [22]:
max_arrivals_by_stop_single = st_prepped.pipe(casf.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=True)
max_arrivals_by_stop_multi = st_prepped.pipe(casf.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=False)

In [23]:
multi_only_explode = casf.get_explode_multiroute_only(max_arrivals_by_stop_single, max_arrivals_by_stop_multi, (HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD))

In [24]:
multi_only_explode.head(3)

,schedule_gtfs_dataset_key,stop_id,route_dir
0,0139b1253130b33adcd4b3a4490530d2,0a2a817a-d35d-434d-b039-6a78d79d7602,91673676-969b-4c0a-a233-158c98c522dc_0
1,0139b1253130b33adcd4b3a4490530d2,0a2a817a-d35d-434d-b039-6a78d79d7602,T6_0
2,0139b1253130b33adcd4b3a4490530d2,52c2636c-34a3-434c-99ae-cdf3dc36d15c,T11x_0


In [27]:
def get_explode_singles(
    single_route_aggregation: pd.DataFrame,
    frequency_thresholds: tuple
) -> pd.DataFrame:
    ms_precursor_threshold = min(frequency_thresholds)
    single_qual = (single_route_aggregation.query('am_max_trips_hr >= @ms_precursor_threshold & pm_max_trips_hr >= @ms_precursor_threshold')
                   .explode('route_dir')
                   .sort_values(['schedule_gtfs_dataset_key','stop_id', 'route_dir'])[['schedule_gtfs_dataset_key','stop_id', 'route_dir']]
                  )
    return single_qual

In [30]:
singles_explode = get_explode_singles(single_frequencies, (HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD)).explode('route_dir')

In [32]:
from tqdm import tqdm
tqdm.pandas()

In [66]:
share_counts = {}

In [67]:
singles_explode.groupby(['schedule_gtfs_dataset_key', 'stop_id']).progress_apply(casf.accumulate_share_count, share_counts=share_counts)

100%|██████████| 19059/19059 [00:03<00:00, 5101.06it/s]


""


In [68]:
qualify_dict = {key: share_counts[key] for key in share_counts.keys() if share_counts[key] >= SHARED_STOP_THRESHOLD}

In [69]:
(singles_explode.query('route_dir.isin(["20-13191_1", "720-13191_1"])').stop_id.value_counts() == 1).value_counts()
#  True: unshared stop count False: shared stop count

True     60
False    16
Name: stop_id, dtype: int64

* local/rapid overlap, there are many unshared stops but that does not offer riders meaningful choice (still a spatial subset)
* need a spatial methodology here...

In [76]:
schedule_gtfs_datasets_to_analyze = np.unique([key.split('__')[0] for key in qualify_dict.keys()])

In [79]:
schedule_gtfs_datasets_to_analyze

array(['0666caf3ec1ecc96b74f4477ee4bc939',
       '3364ec074ca85001da3abd78be2ae521',
       '3c275e5acf8974e1afd765bd3011424c',
       '4b317fc27dde351e12253d46cedd8df0',
       '7cc0cb1871dfd558f11a2885c145d144',
       'c499f905e33929a641f083dad55c521e',
       'ecd018ad66f497fb8f188ed5a71b284b',
       'fb467982dcc77a7f9199bebe709bb700'], dtype='<U32')

In [83]:
helpers.import_scheduled_shapes?

Signature:
helpers.import_scheduled_shapes(
    analysis_date: str,
    filters: tuple = None,
    columns: list = ['shape_array_key', 'geometry'],
    get_pandas: bool = True,
    crs: str = 'EPSG:3310',
) -> Union[geopandas.geodataframe.GeoDataFrame, dask_geopandas.core.GeoDataFrame]
Docstring: Import shapes.
File:      ~/data-analyses/rt_segment_speeds/segment_speed_utils/helpers.py
Type:      function

In [85]:
shapes = helpers.import_scheduled_shapes(analysis_date, columns=['shape_array_key', 'geometry'])

In [87]:
trips = helpers.import_scheduled_trips(analysis_date)

In [89]:
trips.head(3)

,schedule_gtfs_dataset_key,name,trip_id,shape_id,shape_array_key,route_id,route_key,direction_id,route_short_name
0,ff1bc5dde661d62c877165421e9ca257,Santa Ynez Mecatran Schedule,ROUTEA|26003877:T16|17:00:00,ROUTEA:1,ba339972a6377bba13eeca823020f72c,ROUTEA,557edcb059df0484c78588ce8591481e,0.0,None
1,ff1bc5dde661d62c877165421e9ca257,Santa Ynez Mecatran Schedule,ROUTEA|26003877:T9|12:20:00,ROUTEA:1,ba339972a6377bba13eeca823020f72c,ROUTEA,557edcb059df0484c78588ce8591481e,0.0,None
2,ff1bc5dde661d62c877165421e9ca257,Santa Ynez Mecatran Schedule,ROUTEA|26003877:T7|11:10:00,ROUTEA:1,ba339972a6377bba13eeca823020f72c,ROUTEA,557edcb059df0484c78588ce8591481e,0.0,None


* dissolve shapes by route+dir, pick longest
* also try  dissolve shapes by route+dir, dissolve
* find non-overlap by previously qualifying pair